In [2]:
from mxnet import ndarray as nd

a = nd.load('../../../data/mnist.params')
a.keys()

dict_keys(['LeNet_conv0_weight', 'LeNet_conv0_bias', 'LeNet_conv1_weight', 'LeNet_conv1_bias', 'LeNet_dense0_weight', 'LeNet_dense0_bias', 'LeNet_dense1_weight', 'LeNet_dense1_bias'])

In [43]:
import mxnet as mx
pool_attr = {
    'count_include_pad': 'True', 
    'pad': '(0, 0)', 
    'stride': '(2, 2)', 
    'pooling_convention': 'valid', 
    'global_pool': 'False', 
    'pool_type': 'avg', 
    'kernel': '(2, 2)'
}
in_channel = 192
conv_attr = {
    'no_bias': 'True', 
    'dilate': '(1, 1)', 
    'kernel': (2, 2), 
    'stride': '(2, 2)', 
    'pad': '(0, 0)', 
    'layout': 'NCHW', 
    'num_filter': in_channel, 
    'num_group': in_channel,
}
data = mx.sym.var('data')
py = mx.sym.Pooling(data, **pool_attr)
W = mx.sym.var('weight', shape=(in_channel, 1, 2, 2))
cy = mx.sym.Convolution(data, W, **conv_attr)
out = mx.sym.Group([py, cy])
from mxnet import gluon
graph = gluon.nn.SymbolBlock(out, [data])
graph.collect_params().initialize()

In [44]:
weight = nd.full(shape=(in_channel, 1, 2, 2), val=(1/4))
graph.collect_params()['weight'].set_data(weight)

In [70]:
import numpy as np
in_data = np.load('../../../data/densenet/densenet0_conv1_fwd.npy')
out_data = np.load('../../../data/densenet/densenet0_pool1_fwd.npy')
weight_data = np.load('../../../data/densenet/densenet0_pool_conv1_fwd_weight.npy')
conv_out_data = np.load('../../../data/densenet/densenet0_pool_conv1_fwd.npy')
print (in_data.shape, out_data.shape, conv_out_data.shape, weight_data.shape)

(192, 56, 56) (192, 28, 28) (192, 28, 28) (192, 1, 2, 2)


In [74]:
(out_data == conv_out_data).all()

True

In [54]:
a = nd.array(in_data).reshape((1, *in_data.shape))
print (a.shape)
res = graph.forward(a)
print ((res[0].asnumpy() == res[1].asnumpy()).all())
diff = res[0].asnumpy().reshape(out_data.shape) - out_data
assert (diff < 1e-6).all()
print (weight.shape, res[0].shape)

(1, 192, 56, 56)
True
(192, 1, 2, 2) (1, 192, 28, 28)


True